## 分割(交差検証)
- 交差検証(クロスバリデーション)
- データをいくつかに分割して、その分割した1つのデータ群をモデルの評価用のデータとして利用し、その他のデータ群でモデルの学習を行います。
- 過学習の影響を排除して、予測モデルの正確な精度が測定できる方法
- pythonでは機械学習ライブラリであるsklearnライブラリのtrain_test_split関数とKFold関数をよく使う

In [11]:
import numpy as np
import pandas as pd


production_tb = pd.read_csv('data/production.csv', encoding='UTF-8')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#ホールドアウト検証用のデータ分割
#予測モデルの入力値と予測対象の値を別々にtrain_test_split関数に設定

In [12]:
#test_sizeは検証データの割合
train_data, test_data, train_target, test_target = \
train_test_split(production_tb.drop('fault_flg', axis=1),production_tb[['fault_flg']],test_size=0.2)

In [13]:
#train_test_splitによって行名を現在の行番号に直す
train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)

In [14]:
#対象の行番号リストを生成
row_no_list=list(range(len(train_target)))

In [15]:
#交差検証用のデータ分割
k_fold = KFold(n_splits=4, shuffle=True)

#交差数分繰り返し処理、並列処理も可能な部分
for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    
    #交差検証における学習データを抽出
    train_cv = train_data.iloc[train_cv_no, :]
    
    #交差検証における検証データを抽出
    test_cv = train_data.iloc[test_cv_no, :]
    
    #train_dataとtrain_targetを学習データ、
    #test_dataとtest_targetを検証データとして機械学習モデルの構築、検証
    
#交差検証の結果をまとめる
#trainを学習データ、priveate_testを検証データとして機械学習モデルの構築、検証

## 分割(時系列データの検証)

- 時系列データで単純な交差検証は有効ではない。未来データを使って予測モデルを生成している可能性があるため
- Pythonでは時系列データの分割を簡単に実現できるライブラリがないため、自ら実装する必要がある
- 特定のモデルを作る際に自動でデータ分割するような関数を内部で利用すると、モデルを変更するときに都度面倒。
- モデルや利用方法に依存しない汎用的なコードを必ず把握しておく
- 対象のデータセットは月ごとの経営指標です。
- 学習データと検証データを時間軸に対して1か月ごとにスライドしながら生成しましょう
- 学習期間は24ヶ月、検証期間は12ヶ月、スライドする期間は12ヶ月とします

In [16]:
monthly_index_tb = pd.read_csv('data/monthly_index.csv', encoding='UTF-8')

#train_window_startに、最初の学習データの開始行番号を指定
train_window_start = 1

#train_window_endに、最初の学習データの終了行番号を指定
train_window_end = 24

#horizonに検証データのデータ数を指定
horizon = 12

#skipにスライドするデータ数を設定
skip = 12

#年月に基づいてデータを並び替え
monthly_index_tb.sort_values(by='year_month')

while True:
    #検証データの終了行番号を計算
    test_window_end = train_window_end + horizon
    
    #行番号を指定して、元データから学習データを取得
    #train_window_startの部分を1に固定すれば学習データを増やしていく検証に変更可能
    train = monthly_index_tb[train_window_start:train_window_end]
    
    #行番号を指定して元データから検証データを取得
    test=monthly_index_tb[(train_window_end + 1):test_window_end]
    
    #検証データの終了行番号が元データの行数以上になっているか判定
    if test_window_end >= len(monthly_index_tb.index ): 
        #全データを対象にした場合終了
        break
    
    #データをスライドさせる
    train_window_start += skip
    train_window_end += skip
    
    #交差結果をまとめる


## データ生成

- 学習データが不均衡な場合に予測精度が下がることが多い
- 解消方法としては機械学習モデル側で重みを与えること
- もう一つはデータを操作して不均衡を解消する方法で以下がある
- (1)少ないデータを増やすオーバーサンプリング
- (2)多いデータを減らすアンダーサンプリング

## アンダーサンプリング

- データの選択方法は重複選択(同じデータを2回以上選択)なしがおすすめ
- サンプリング数が少ない場合に偏りのないランダムサンプリングを実現したい場合
- →事前にデータをクラスタリングし、作成されたクラスタごとにサンプリングをすると良い
- ただ、アンダーサンプリングはデータを間引いて情報量を少なくするのであまりよくない
- よく使うのは少ないデータをオーバーサンプリングし、多いデータをアンダーサンプリングをする

## オーバーサンプリング

-  SMOTEを使う(アルゴリズム概要は以下)
- 1.生成元のデータからランダムに一つのデータを選択
- 2.設定したkの値を元に、1-kの整数値(一様分布)からランダムに選択肢nを設定
- 3.1.で選択したデータにn番目に近いデータを新たに選択
- 4.1と3で選択したデータを元に新たなデータを生成
- 製造レコードで障害が起きていないレコード927件と障害が起きているレコード73件がある。
- 障害が起きているレコードをSMOTEを用いてオーバーサンプリングを行い、障害が起きていないレコードの件数に近づける

In [23]:
# SMOTE関数をライブラリから読み込み
from imblearn.over_sampling import SMOTE

#SMOTE関数の設定
#ratioは不均衡データにおける少ない例のデータを多い方のデータの何割まで増やすか設定
#(autoの場合は同じ数まで増やす、0.5と設定すると5割までデータを増やす)
#k_neighborsはsmoteのkパラメータ
#random_stateは乱数のseed(乱数の生成パターンの元)
sm=SMOTE(ratio='auto', k_neighbors=5,random_state=71)

#オーバーサンプリング実行
balance_data, balance_target = sm.fit_sample(production_tb[['length',  'thickness']], production_tb['fault_flg'])

## テーブル横持ち・縦持ち

-  データがレコード形式になっている場合を縦持ち、データが表形式の状態を横持ちという。
- 縦持ちのデータを作成した後に、横持ちにデータを変換する必要がある
- データセットはホテルの予約レコードです。予約テーブルから、顧客/宿泊人数ごとに予約数をカウントし、
- 行を顧客ID、列を宿泊人数、値を予約数の行列に変換しましょう
- pivot_table関数は一つ目の引数に対象テーブル、2つめにkeyとなるindex、columnsにデータ要素の種類
- values引数にデータ要素の値となる対象の列を指定
- aggfunc引数にvalues引数で指定された列値をデータの要素の値に変換する関数を指定します。

In [28]:
#pivot_table関数で、横持ち変換と集約処理を同時実行
#aggfuncに予約数をカウントする関数を指定
reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')
pivot_reserve_tb=pd.pivot_table(reserve_tb, index='customer_id', columns='people_num', values='reserve_id', aggfunc=lambda x: len(x), fill_value=0)
pivot_reserve_tb.head()

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1


## スパースマトリックスへの変換

-  ほとんど要素の値が0の疎行列では横持ちすると列数が多くなるので、データサイズが膨れないようにする必要がある

In [37]:
#スパースマトリックスのライブラリを読み込み
from scipy.sparse import csc_matrix
reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')

#顧客ID/宿泊人数別の予約数の表を生成
cnt_tb =reserve_tb.groupby(['customer_id', 'people_num'])['reserve_id'].size().reset_index()
cnt_tb.columns=['customer_id', 'people_num', 'rsv_cnt']

#sparseMatrixの行/列に該当する列の値をカテゴリ型に変換
#カテゴリ型については「第9章 カテゴリ型」で詳しく説明

customer_id = pd.Categorical(cnt_tb['customer_id'])
people_num = pd.Categorical(cnt_tb['people_num'])

#スパースマトリックスを生成
#1の引数は指定した行列に対応した値、行番号、列番号の配列をまとめたタプルを指定
#shapeにはスパースマトリックスのサイズを指定
#customer_id.codesはインデックス番号の取得
#(len(customer_id.categories))はcustomer_idのユニークな数を取得)
csc_matrix((cnt_tb['rsv_cnt'], (customer_id.codes, people_num.codes)),
           shape=(len(customer_id.categories), len(people_num.categories)))


<888x4 sparse matrix of type '<class 'numpy.int64'>'
	with 2366 stored elements in Compressed Sparse Column format>